# Model training done on kaggle notebook with GPU T4 x2.

In [1]:
!pip install tensorflow
!pip install matplotlib
!pip install wurlitzer
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


In [3]:
import tensorflow as tf
import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import os

In [7]:
# Check if GPU is available
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU not available")

GPU is available


In [ ]:
BATCH_SIZE = 64
IMG_SIZE = (224, 224)

In [ ]:
def load_data(train_dir, batch_size, img_size):
    # Data augmentation for training data
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        rotation_range=30,
        zoom_range=0.3,
        width_shift_range=0.3,
        height_shift_range=0.3,
        brightness_range=[0.7, 1.3]
    )

    # Only rescale for validation data
    val_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2
    )

    train_ds = train_datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )

    val_ds = val_datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )

    return train_ds, val_ds

In [ ]:
def create_model(fine_tune_at=None):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    if fine_tune_at is not None:
        base_model.trainable = True
        # Freeze all the layers before the `fine_tune_at` layer
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False
    else:
        base_model.trainable = False  # Initially, freeze the base model

    # Correctly handle the output tensor from the base model
    x = base_model.output
    if isinstance(x, list):
        print("It's a list")
        x = x[0]  # Take the first tensor if the output is a list
    x = Flatten()(x)
    
    """
    Add more layers if the data is very huge.
    x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)  # Additional dense layer with L2 regularization
    x = Dropout(0.3)(x)  # Dropout to reduce overfitting
    x = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = Dropout(0.3)(x)
    """
    
    output = Dense(2, activation='softmax')(x)  # 2 output classes for binary classification

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
def train_and_fine_tune_model(model, train_ds, val_ds):
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    checkpoint = ModelCheckpoint('/kaggle/working/bio_nonbio_model_best.keras', save_best_only=True, monitor='val_loss', mode='min', save_format='keras')
    history = model.fit(train_ds, epochs=50, validation_data=val_ds, callbacks=[early_stopping, checkpoint])
   
    return history

In [ ]:
def evaluate_model(model, test_ds):
    loss, accuracy = model.evaluate(test_ds, verbose=2)
    print(f"Testing set Accuracy: {accuracy}")
    return accuracy

In [ ]:
def plot_history(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.show()

In [ ]:
def compute_confusion_matrix(model, test_ds):
    # Predict the values from the test dataset
    predictions = model.predict(test_ds)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_ds.classes
    class_labels = list(test_ds.class_indices.keys())
    
    # Print classification report
    print("Classification Report:\n", classification_report(true_classes, predicted_classes, target_names=class_labels))
    
    # Compute confusion matrix
    cm = confusion_matrix(true_classes, predicted_classes)
    print("Confusion Matrix:\n", cm)

In [ ]:
TRAIN_DIR = "/kaggle/input/non-and-biodegradable-waste-dataset/TRAIN.1"

train_ds, val_ds = load_data(TRAIN_DIR, BATCH_SIZE, IMG_SIZE)

for i in (train_ds.class_indices):
    print(i)

# Fine-tuning the last 4 layers
model = create_model(fine_tune_at=15)
history = train_and_fine_tune_model(model, train_ds, val_ds)
evaluate_model(model, val_ds)
plot_history(history)

# Compute and print confusion matrix
compute_confusion_matrix(model, val_ds)

# **Let's Test the Model**

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from PIL import Image
import io

# Load the model
model = tf.keras.models.load_model("/kaggle/input/waste-classification-model/tensorflow2/default/1/bio_nonbio_model_best.keras")

# Let's see if the model loads by checking the summary
print(model.summary())

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [17]:
# Class labels
class_labels = ["Biodegradable", "Non-Biodegradable"]

def preprocess_image(img_path):
    img = Image.open(img_path)
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Path to the image you want to predict
image_path = "/kaggle/input/test-data/Tests/test9.jpeg"

# Preprocess the image
img_array = preprocess_image(image_path)

# Make the prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)[0]
predicted_label = class_labels[predicted_class]
confidence = predictions[0][predicted_class]

print(f"Predicted class: {predicted_label}, Confidence: {confidence:.2f}")


1/1 [==============================] - 0s 293ms/step
Predicted class: Non-Biodegradable, Confidence: 1.00


# Let's convert to TensorFlow.js

In [7]:
!pip install tensorflowjs

INFO: pip is looking at multiple versions of tensorstore to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of tensorstore to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 26.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: tensorstore
    Found existing installation: tensorstore 0.1.63
    Uninstalling tensorstore-0.1.63:
      

In [8]:
import tensorflowjs as tfjs

print(tfjs.__version__)

4.20.0


**First, we need to save it as .h5 because tf_js converter doesn't support .keras model file

Or just save it as .h5 in the first place**

In [18]:
# Save the model in HDF5 format
model.save('/kaggle/working/bio_nonbio_model_best.h5', save_format='h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


If the conversion doesn't work try from command line with the following commad:

!tensorflowjs_converter --input_format=keras /kaggle/working/bio_nonbio_model_best.h5 /kaggle/working/tfjs_model

In [ ]:
# Path to the Keras model
keras_model_path = "/kaggle/input/waste-classification-model/tensorflow2/default/1/bio_nonbio_model_best.keras"

# Destination path for the TensorFlow.js model
tfjs_target_dir = "/kaggle/working/bio_nonbio_model_best_tfjs_model"

# Convert the model
tfjs.converters.save_keras_model(tf.keras.models.load_model(keras_model_path), tfjs_target_dir)